# Surface pre-processing.

This should be quite straighforward.

1. Denoising of data
2. Averaging over runs.

I am going to try and rely solely on python-based tools for this.

## First, get the set of gifti_files.

In [1]:
funcpath='/Volumes/BAHAMUT/PRF_DATA/ds002574-download/derivatives/fmriprep/sub-02/ses-01/func'


def findgiftis(FOLDER,ext):
    
    # Function goal: return nifti files within a directory
    
    # Detail: could be used for a file with any extension...
    
    #Inputs
    
    # FOLDER (directory to search)
    # ext (the suffix (.nii or .nii.gz))
    
    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)
    import os
    import sys
    niftilist=list()
    for file in os.listdir(FOLDER):
        if file.endswith(ext):
            niftilist.append(FOLDER+"/"+file)
    fprintf(sys.stdout, "Found %d gifti files \n", len(niftilist))
    return(sorted(niftilist))

In [2]:
base_folder='/Volumes/BAHAMUT/ALL_PRF_DAT/BIGRUN/derivatives/fmriprep/sub-02'

In [4]:
import os

def listabsdir(FOLDER,ext):
    
   
    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)
    import os
    import sys
    niftilist=list()
    for file in os.listdir(FOLDER):
        if not file.startswith('.') and file.startswith(tuple(ext)):
            niftilist.append(FOLDER+"/"+file)
    fprintf(sys.stdout, "Found %d directories \n", len(niftilist))
    return(sorted(niftilist))


dirs=listabsdir(base_folder,'ses')

Found 4 directories 


In [3]:
funcgiftsw=list()

for cdir in dirs:
    funcgiftsw.append(findgiftis(cdir+'/func','.gii'))

NameError: name 'dirs' is not defined

In [5]:
len(funcgiftsw[0])

52

In [6]:
funcgifts = []
for sublist in funcgiftsw:
    for item in sublist:
        funcgifts.append(item)

In [7]:
len(funcgifts)

184

In [8]:
leftgifts=[nift for nift in funcgifts if "fsaverage_hemi-L" in(nift)]
rightgifts=[nift for nift in funcgifts if "fsaverage_hemi-R" in(nift)]

In [9]:
len(leftgifts)

46

We have a gifti file for each hemisphere and for each run.

In [10]:
import nilearn
from nilearn import surface

## First, detrend the data using a savgolay filter.

In [11]:
def savgol_filter(data, polyorder=3, deriv=0, window_length = 120,tr =1.5):
    """ Applies a savitsky-golay filter to a nifti-file.

    Fits a savitsky-golay filter to a 4D fMRI nifti-file and subtracts the
    fitted data from the original data to effectively remove low-frequency
    signals.

    Parameters
    ----------
    in_file : str
        Absolute path to nifti-file.
    polyorder : int (default: 3)
        Order of polynomials to use in filter.
    deriv : int (default: 0)
        Number of derivatives to use in filter.
    window_length : int (default: 120)
        Window length in seconds.

    Returns
    -------
    out_file : str
        Absolute path to filtered nifti-file.
    """

    import nibabel as nib
    from scipy.signal import savgol_filter
    import numpy as np
    import os

    dims = data.shape
    

    # TR must be in seconds
    if tr < 0.01:
        tr = np.round(tr * 1000, decimals=3)
    if tr > 20:
        tr = tr / 1000.0

    window = np.int(window_length / tr)
    
    # Window must be odd
    if window % 2 == 0:
        window += 1

    data = data.reshape((np.prod(data.shape[:-1]), data.shape[-1]))
    data_filt = savgol_filter(data, window_length=window, polyorder=polyorder,
                              deriv=deriv, axis=1, mode='nearest')

    data_filt = data - data_filt + data_filt.mean(axis=-1)[:, np.newaxis]
    data_filt = data_filt.reshape(dims)

    return data_filt

In [12]:
polys=3
deriv=0
wl=120
tr=1.5

In [13]:
from joblib import Parallel, delayed

In [14]:
cleaned_data_L=Parallel(n_jobs=5,verbose=9)(delayed(savgol_filter)(surface.load_surf_data(fn),polys,deriv,wl,tr)  for fn in leftgifts)
cleaned_data_R=Parallel(n_jobs=5,verbose=9)(delayed(savgol_filter)(surface.load_surf_data(fn),polys,deriv,wl,tr)  for fn in rightgifts)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   9 tasks      | elapsed:   27.8s
[Parallel(n_jobs=5)]: Done  23 tasks      | elapsed:   55.2s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:  1.3min remaining:    5.5s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:  1.3min finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   24.1s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   48.5s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:  1.3min remaining:    5.4s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:  1.3min finished


## Nuissance regression

Get nuissance files

In [15]:
nfilesw=list()

    
    
def listabsdir(FOLDER,ext):
    
   
    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)
    import os
    import sys
    niftilist=list()
    for file in os.listdir(FOLDER):
        if not file.startswith('.') and file.endswith(tuple(ext)):
            niftilist.append(FOLDER+"/"+file)
    fprintf(sys.stdout, "Found %d directories \n", len(niftilist))
    return(sorted(niftilist))


for cdir in dirs:
    nfilesw.append(listabsdir(cdir+'/func','.tsv'))

Found 13 directories 
Found 8 directories 
Found 10 directories 
Found 15 directories 


In [16]:
n_files = []
for sublist in nfilesw:
    for item in sublist:
        n_files.append(item)

In [17]:
n_components =5 # number of pca components to regress out

# grab these variables from nuissance file:
n_components=5
varr = ['std_dvars','dvars','framewise_displacement','a_comp_cor_00','a_comp_cor_01','a_comp_cor_02',
    'a_comp_cor_03',
    'a_comp_cor_04',
    'a_comp_cor_05',
    'trans_x',
    'trans_y',
    'trans_z',
    'rot_x',
    'rot_y',
    'rot_z',
    'cosine00']

Now apply the same filter to the nuissance variables.

In [18]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from nistats.regression import OLSModel

def savgol_filter(data, polyorder=3, deriv=0, window_length = 120,TR=1.5):
    """ Applies a savitsky-golay filter to a nifti-file.

    Fits a savitsky-golay filter to a 4D fMRI nifti-file and subtracts the
    fitted data from the original data to effectively remove low-frequency
    signals.

    """

    from scipy.signal import savgol_filter

    window = np.int(window_length / TR)

    # Window must be odd
    if window % 2 == 0:
        window += 1

    data_filt = savgol_filter(data, window_length=window, polyorder=polyorder,
                              deriv=deriv, mode='nearest')

    data_filtered = data - data_filt + data_filt.mean()

    return data_filtered


def perform_ng(data,nuissance,varr,n_components):
    df = pd.read_csv(nuissance, sep='\t', header=0,index_col=None)
    # get the wanted variables and do stuff with it
    nuissances = []
    for var in varr:
        # get wanted nuissance variables
        ndat = np.array(df[var])
        # fill in nans
        ndat[ndat=='n/a'] = np.nan
        # cast to float
        ndat = ndat.astype('float32')
            # median fill nan values (i.e. first value )
        ndat[np.isnan(ndat)] = np.nanmedian(ndat)
            # temporally filter 
        filtered_data = savgol_filter(ndat)
            # z-score (so that explained variance ratios is interpretable)
        filtered_data_z = (filtered_data - np.mean(filtered_data)) / np.std(filtered_data)
            # and append
        nuissances.append(filtered_data_z)

    nuissances = np.array(nuissances)    

        # now do pca and grab first 5:
    pca = PCA(n_components=n_components)  
    pcas = pca.fit_transform(nuissances.T)

    datashape = data.shape

        # do nuissance regression
    dm = np.hstack([np.ones((pcas.shape[0],1)),pcas]) # add intercept
    model = OLSModel(dm)
    fit = model.fit(data.reshape(-1,datashape[-1]).T)
    resid = fit.resid.T.reshape(datashape)
    resid += np.mean(data,axis=-1)[:,np.newaxis] # re-add the signal offset which was regressed out by the intercept
    return resid, fit.theta,fit.r_square

/Users/nicholashedger/anaconda2/envs/py3env/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  after removing the cwd from sys.path.


In [19]:
denoised_data_L=Parallel(n_jobs=5,verbose=9)(delayed(perform_ng)(cleaned_data_L[fn],n_files[fn],varr,n_components)  for fn in range(46))

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   14.4s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   31.0s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:   56.9s remaining:    4.0s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:   58.7s finished


In [20]:
denoised_data_R=Parallel(n_jobs=5,verbose=9)(delayed(perform_ng)(cleaned_data_R[fn],n_files[fn],varr,n_components)  for fn in range(46))

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:   19.0s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   35.9s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:  1.0min remaining:    4.2s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:  1.0min finished


In [22]:
from nilearn import plotting

surfpath='/Volumes/BAHAMUT/ALL_PRF_DAT/BIGRUN/derivatives/freesurfer/fsaverage/surf'

surface_files=listabsdir(surfpath, ['pial','orig','sphere', 'white','inflated','sulc','curv'])
surface_filesL=[surf for surf in surface_files if "lh" in(surf)]
surface_filesR=[surf for surf in surface_files if "rh" in(surf)]


flatlh='/Volumes/BAHAMUT/ALL_PRF_DAT/BIGRUN/derivatives/freesurfer/fsaverage/surf/flat_lh.gii'
flatrh='/Volumes/BAHAMUT/ALL_PRF_DAT/BIGRUN/derivatives/freesurfer/fsaverage/surf/flat_rh.gii'

Found 19 directories 


In [21]:
R2sL=[]
for bits in denoised_data_L:
    R2sL.append(np.nan_to_num(bits[2],0))
    
R2sR=[]
for bits in denoised_data_R:
    R2sR.append(np.nan_to_num(bits[2],0))

    
    
R2sLav=np.mean(R2sL,axis=0)
R2sRav=np.mean(R2sR,axis=0)

In [26]:
R2array=np.concatenate([R2sLav,R2sRav],axis=0)
R2array.shape

(327684,)

In [27]:
import nibabel

outpath='/Volumes/BAHAMUT/OUTPUT/FULL'

nibabel.freesurfer.io.write_morph_data(os.path.join(outpath,'noisefit_curv'),np.squeeze(R2array))


In [24]:
del cleaned_data_L, cleaned_data_R

In [25]:
mydataL=[]
for bits in denoised_data_L:
    mydataL.append(bits[0])
    
mydataR=[]
for bits in denoised_data_R:
    mydataR.append(bits[0])

In [ ]:
#viewang=plotting.view_surf(surf_mesh=flatlh,surf_map=R2sLav,bg_map=surface_filesL[1],symmetric_cmap=False,threshold=1e-6,cmap='plasma',title='R2 of noise model')
#viewang

In [ ]:
#viewang=plotting.view_surf(surf_mesh=flatrh,surf_map=R2sRav,bg_map=surface_filesR[1],symmetric_cmap=False,threshold=1e-6,cmap='plasma',title='R2 of noise model')
#viewang

## Compute tsnrs

In [26]:
def compute_tsnr(data):
    tsnr = np.mean(data,axis=-1)/np.std(data,axis=-1)
    tsnr[np.isnan(tsnr)] = 0
    tsnr[np.isinf(tsnr)] = 0
    tsnr += np.abs(np.min(tsnr))+0.01
    tsnr=np.squeeze(tsnr)
    return(tsnr)

In [27]:
tsnr_L=Parallel(n_jobs=5,verbose=9)(delayed(compute_tsnr)(mydataL[fn])  for fn in range(46))
tsnr_R=Parallel(n_jobs=5,verbose=9)(delayed(compute_tsnr)(mydataR[fn])  for fn in range(46))
tsnr_L_arr=np.asarray(tsnr_L)
tsnr_R_arr=np.asarray(tsnr_R)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    4.0s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   13.5s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:   26.9s remaining:    1.9s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:   28.4s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    5.8s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:   12.7s
[Parallel(n_jobs=5)]: Done  43 out of  46 | elapsed:   24.4s remaining:    1.7s
[Parallel(n_jobs=5)]: Done  46 out of  46 | elapsed:   25.7s finished


In [28]:
import dill
def dill_save(obj,filename):
    with open(filename,'wb') as output:
        dill.dump(obj,output)
        
outputpath='/Volumes/BAHAMUT/TEMP'


dill_save(tsnr_L_arr,os.path.join(outputpath,'tsnr_l.pkl'))
dill_save(tsnr_R_arr,os.path.join(outputpath,'tsnr_r.pkl'))

## Convert to percent signal change.

In [27]:
psc_L=list()
psc_R=list()


for fp in range(46):
    psc_L.append(nilearn.signal.clean(signals=np.transpose(mydataL[fp]), detrend=False, standardize='psc', low_pass=None, high_pass=None, t_r=1.5, ensure_finite= True))
    psc_R.append(nilearn.signal.clean(signals=np.transpose(mydataR[fp]), detrend=False, standardize='psc', low_pass=None, high_pass=None, t_r=1.5, ensure_finite= True))

/Users/nicholashedger/anaconda2/envs/py3env/lib/python3.6/site-packages/nilearn/signal.py:77: RuntimeWarning: invalid value encountered in true_divide
  signals = (signals / mean_signal) * 100
/Users/nicholashedger/anaconda2/envs/py3env/lib/python3.6/site-packages/nilearn/signal.py:81: UserWarning: psc standardization strategy is meaningless for features that have a mean of 0 or less. These time series are set to 0.
  warnings.warn('psc standardization strategy is meaningless '


In [31]:
import dill
def dill_save(obj,filename):
    with open(filename,'wb') as output:
        dill.dump(obj,output)

In [32]:
outputpath='/Volumes/BAHAMUT'

In [33]:
dill_save(psc_L,os.path.join(outputpath,'l.pkl'))
dill_save(psc_R,os.path.join(outputpath,'r.pkl'))

## Average across runs

In [ ]:
#def runav(data,tsnr):
#    dmean=np.squeeze(np.average(data,axis=0,weights=tsnr))
#    return dmean

def runav(data,tsnr):
    dmean=np.squeeze(np.average(data,axis=0,weights=tsnr))
    return dmean

Perform this one tr at a time (this is to avoid memory issues)

In [ ]:
meanlist_L=Parallel(n_jobs=5,verbose=9)(delayed(runav)(psc_L_arr[:,fn,:],tsnr_L_arr)  for fn in range(120))
meanlist_R=Parallel(n_jobs=5,verbose=9)(delayed(runav)(psc_R_arr[:,fn,:],tsnr_R_arr)  for fn in range(120))

In [ ]:
meanvolL=np.asarray(meanlist_L)
meanvolR=np.asarray(meanlist_R)

In [ ]:
meanvolL.shape

The resulting shape is trs x vertices

In [ ]:
atlaspath='/Volumes/BAHAMUT/ATLAS/lh.wang2015_atlas.mgz'
surfpath='/Volumes/BAHAMUT/PRF_DATA/ds002574-download/derivatives/freesurfer/fsaverage/surf'
atlas = surface.load_surf_data(atlaspath)

In [ ]:
import os
def listabsdir(FOLDER,ext):
    
   
    def fprintf(stream, format_spec, *args):
        stream.write(format_spec % args)
    import os
    import sys
    niftilist=list()
    for file in os.listdir(FOLDER):
        if not file.startswith('.') and file.endswith(tuple(ext)):
            niftilist.append(FOLDER+"/"+file)
    fprintf(sys.stdout, "Found %d files \n", len(niftilist))
    return(sorted(niftilist))

In [ ]:
from nilearn import plotting
surface_files=listabsdir(surfpath, ['pial','orig','sphere', 'white','inflated'])
surface_filesL=[surf for surf in surface_files if "lh" in(surf)]
surface_filesR=[surf for surf in surface_files if "rh" in(surf)]

plotting.view_surf(surface_filesL[4],surf_map=atlas)

In [ ]:
roinames=["None", "V1v", "V1d" ,"V2v" ,"V2d" ,"V3v" ,"V3d" ,"hV4" ,"VO1" ,"VO2" ,"PHC1" ,"PHC2",
    "TO2" ,"TO1" ,"LO2", "LO1" ,"V3B","V3A","IPS0","IPS1","IPS2","IPS3","IPS4",
    "IPS5" ,"SPL1" ,"FEF"]

Get the vertices that correspond to V1.

In [ ]:
number=roinames.index("V1d")
number

In [ ]:
idx=np.squeeze(np.where(atlas == number))

In [ ]:
signalsL=meanvolL[:,idx]
signalsR=meanvolR[:,idx]

Plot the data within V1

In [ ]:
figsizex=70
figsizey=10

import matplotlib.pyplot as plt

def plotts(data):
    ylim=(np.min(data),np.max(data))
    plt.figure(figsize=(figsizex,figsizey))
    plt.plot(data)
    plt.rcParams.update({'font.size': 40})
    plt.ylim(ylim)
    plt.xlabel('Time')
    plt.ylabel('Response')
    plt.show()

plotts(meanvolL[:,idx])

In [ ]:
plotts(meanvolL[:,idx[2]])

## Save the mean volumes for model fitting.

In [ ]:
dumpdir='/Volumes/BAHAMUT/DUMP'

In [ ]:
import dill
import os

def dill_save(obj,filename):
    with open(filename,'wb') as output:
        dill.dump(obj,output)

dill_save(meanvolL,os.path.join(dumpdir,'LHEM.pkl'))
dill_save(meanvolR,os.path.join(dumpdir,'RHEM.pkl'))

In [ ]:
idx